<a href="https://colab.research.google.com/github/XG11/ECE420FinalProject/blob/main/ECE420Phase3OpenCV%2BSVMmethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.ndimage import gaussian_filter, gaussian_gradient_magnitude
from scipy.spatial.distance import euclidean
from scipy.signal import convolve2d
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
from PIL import Image
import numpy as np

uploaded_images1 = files.upload()
uploaded_images2 = files.upload()

image_path1 = next(iter(uploaded_images1))
image_path2 = next(iter(uploaded_images2))

image1 = Image.open(image_path1).convert('L')
image2 = Image.open(image_path2).convert('L')

image1_np = np.array(image1, dtype=np.float32)
image2_np = np.array(image2, dtype=np.float32)


StopIteration: 

In [2]:
from google.colab import files
from PIL import Image
import numpy as np

# Upload multiple images at once
uploaded_images = files.upload()

# Initialize a list to store images
images_np = []

# Iterate over the uploaded files
for image_path in uploaded_images.keys():
    # Open the image, convert it to grayscale, and convert to numpy array
    image = Image.open(image_path).convert('L')
    image_np = np.array(image, dtype=np.float32)
    images_np.append(image_np)

# Now images_np contains all the uploaded images as numpy arrays
print(f'Uploaded {len(images_np)} images.')


Saving s7.jpg to s7.jpg
Saving s6.jpg to s6.jpg
Saving s5.jpg to s5.jpg
Saving s4.jpg to s4.jpg
Saving s3.jpg to s3.jpg
Saving s2.jpg to s2.jpg
Saving s1.jpg to s1.jpg
Saving n5.jpg to n5.jpg
Saving n4.jpg to n4.jpg
Saving n3.jpg to n3.jpg
Saving n2.jpg to n2.jpg
Saving n1.jpg to n1.jpg
Saving imge12.jpg to imge12.jpg
Saving imge11.jpg to imge11.jpg
Saving imge10.jpg to imge10.jpg
Saving imge9.jpg to imge9.jpg
Saving imge8.jpg to imge8.jpg
Saving imge7.jpg to imge7.jpg
Saving imge6.jpg to imge6.jpg
Saving imge5.jpg to imge5.jpg
Saving imge4.jpg to imge4.jpg
Saving imge3.jpg to imge3.jpg
Saving imge2.jpg to imge2.jpg
Saving imge1.jpg to imge1.jpg
Saving img10.jpg to img10.jpg
Saving img9.jpg to img9.jpg
Saving img8.jpg to img8.jpg
Saving img7.jpg to img7.jpg
Saving img6.jpg to img6.jpg
Saving img5.jpg to img5.jpg
Saving img4.jpg to img4.jpg
Saving img3.jpg to img3.jpg
Saving img2.jpg to img2.jpg
Saving img1.jpg to img1.jpg
Uploaded 34 images.


In [ ]:
'''import cv2
import numpy as np
from google.colab.patches import cv2_imshow

img1 = cv2.imread('Stop1.jpg', cv2.IMREAD_GRAYSCALE)  # Query image
img2 = cv2.imread('stop2.jpg', cv2.IMREAD_GRAYSCALE)  # Train image


# detect SIFT key points and descriptors
sift = cv2.SIFT_create()

# Find the key points and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# Step 2: Use BFMatcher to match descriptors
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)  # L2 norm (Euclidean distance)

# Use KNN to find the best matches
matches = bf.knnMatch(des1, des2, k=2)

# Step 3: Apply Lowe's ratio test to filter good matches
good_matches = []
for m, n in matches:
    if m.distance <  n.distance:  # Lowe's ratio test
        good_matches.append(m)

# Step 4: Draw the matches
img_matches = cv2.drawMatches(
    img1, kp1, img2, kp2, good_matches, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)


# Step 5: Display the result
cv2_imshow(img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()


'''

"import cv2\nimport numpy as np\nfrom google.colab.patches import cv2_imshow\n\nimg1 = cv2.imread('Stop1.jpg', cv2.IMREAD_GRAYSCALE)  # Query image\nimg2 = cv2.imread('stop2.jpg', cv2.IMREAD_GRAYSCALE)  # Train image\n\n\n# detect SIFT key points and descriptors\nsift = cv2.SIFT_create()\n\n# Find the key points and descriptors with SIFT\nkp1, des1 = sift.detectAndCompute(img1, None)\nkp2, des2 = sift.detectAndCompute(img2, None)\n\n# Step 2: Use BFMatcher to match descriptors\nbf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)  # L2 norm (Euclidean distance)\n\n# Use KNN to find the best matches\nmatches = bf.knnMatch(des1, des2, k=2)\n\n# Step 3: Apply Lowe's ratio test to filter good matches\ngood_matches = []\nfor m, n in matches:\n    if m.distance <  n.distance:  # Lowe's ratio test\n        good_matches.append(m)\n\n# Step 4: Draw the matches\nimg_matches = cv2.drawMatches(\n    img1, kp1, img2, kp2, good_matches, None,\n    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS\n)\n\

In [4]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


img1 = cv2.imread('Stop1.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('stop2.jpg', cv2.IMREAD_GRAYSCALE)


#create SIFT object
sift = cv2.SIFT_create()
#put in training images into this variable
image_paths_train = ['img2.jpg','img3.jpg','img4.jpg','img5.jpg','img6.jpg','img7.jpg','img8.jpg','img9.jpg','n1.jpg','n2.jpg','n3.jpg','n4.jpg','n5.jpg','s1.jpg', 's2.jpg', 's3.jpg', 's4.jpg', 's5.jpg', 's6.jpg', 's7.jpg']
# Find descriptors for each image and append to descriptor train vector
descriptors_train = []
for img_path in image_paths_train:
  image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  keypoints, descriptors = sift.detectAndCompute(image, None)
  descriptors_train.append(descriptors)
#size(descriptors_train) = #keypoints in each image added up
descriptors_train = np.vstack(descriptors_train)
#Once you gather All descriptors Perform Kmeans Algorithm
vocabulary_size = 500
kmeans = KMeans(n_clusters=vocabulary_size, random_state=0)
kmeans.fit(descriptors_train)

centroids_arr = kmeans.cluster_centers_
#put in test images into this variable
image_paths_test = ['imge1.jpg','imge2.jpg','imge3.jpg','imge4.jpg','imge5.jpg','imge6.jpg','imge7.jpg','imge8.jpg','imge9.jpg','imge10.jpg','imge11.jpg','imge12.jpg','img5.jpg']

hist_arr_train = [] #stores histograms for all training images

hist_arr_test = [] #stores histograms for all test images

#Find descriptors for each image and append to descriptor test vector
#making histograms for training images
for img_path in image_paths_train:
  image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  keypoints, descriptors = sift.detectAndCompute(image, None)
  k_means_labels = kmeans.predict(descriptors)
  hist = np.zeros(vocabulary_size)
  for label in k_means_labels:
    hist[label] += 1
  hist /= hist.sum()
  hist_arr_train.append(hist)


#making histograms for test images
for img_path in image_paths_test:
  image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  keypoints, descriptors = sift.detectAndCompute(image, None)
  k_means_labels = kmeans.predict(descriptors)
  hist = np.zeros(vocabulary_size)
  for label in k_means_labels:
    hist[label] += 1
  hist /= hist.sum()
  hist_arr_test.append(hist)


#SVM Classification
X_train = np.array(hist_arr_train).astype(np.float32)


svm_classifier = cv2.ml.SVM_create()
svm_classifier.setType(cv2.ml.SVM_C_SVC)
svm_classifier.setKernel(cv2.ml.SVM_POLY)
svm_classifier.setC(1)
svm_classifier.setGamma(1/X_train.shape[1])
svm_classifier.setDegree(3)
svm_classifier.setCoef0(0)




y_train = np.ones(len(image_paths_train)).astype(np.int32)
y_train[-12:] = 0

svm_classifier.train(X_train, cv2.ml.ROW_SAMPLE, y_train)
svm_classifier.save('svm_model.xml')

X_test = np.array(hist_arr_test).astype(np.float32)

y_test = np.zeros(len(image_paths_test)).astype(np.int32)
y_test[len(image_paths_test) - 1] = 1

predicted_labels = svm_classifier.predict(X_test)[1]
predicted_labels = predicted_labels.flatten()  # Reshape to a 1D array
print(predicted_labels)
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")




[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Accuracy: 92.31%


In [ ]:


#size(descriptors_test) = #keypoints in each image added up
# kp1, des1 = sift.detectAndCompute(img1, None)
# kp2, des2 = sift.detectAndCompute(img2, None)

# print(len(kp1))
# print(len(kp2))

# # BFMatcher to match descriptors
# bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)  # L2 norm (Euclidean distance)

# #find the best matches
# matches = bf.knnMatch(des1, des2, k=2)

# #ratio test to filter good matches
# good_matches = []
# for m, n in matches:
#     if m.distance <  n.distance:  # Lowe's ratio test
#         good_matches.append(m)

# #matches
# img_matches = cv2.drawMatches(
#     img1, kp1, img2, kp2, good_matches, None,
#     flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
# )

# plt.figure(figsize=(12, 6))


# plt.subplot(1, 2, 1)
# plt.imshow(img1, cmap='gray')
# plt.scatter([kp.pt[0] for kp in kp1], [kp.pt[1] for kp in kp1], c='r', s=10)
# plt.title("Image 1 Keypoints")


# plt.subplot(1, 2, 2)
# plt.imshow(img2, cmap='gray')
# plt.scatter([kp.pt[0] for kp in kp2], [kp.pt[1] for kp in kp2], c='r', s=10)
# plt.title("Image 2 Keypoints")


# plt.show()
# cv2_imshow(img_matches)
# cv2.waitKey(0)
# cv2.destroyAllWindows()